In [1]:
from trainer import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=42)
train_df

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
1006,2011,88.96,m,v,as,c,d,ag,k,x,...,0,0,1,0,0,0,0,0,0,0
1840,3690,89.90,n,s,as,d,d,ae,g,s,...,1,0,0,0,0,0,0,0,0,0
3792,7597,92.59,f,c,m,c,d,v,i,e,...,0,0,1,0,0,0,0,0,0,0
152,322,108.84,j,aa,g,d,d,i,i,e,...,0,1,0,0,0,0,0,0,0,0
1557,3103,111.15,ay,i,as,c,d,ad,l,k,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,6879,109.42,j,i,as,c,d,r,l,r,...,0,0,1,0,0,0,0,0,0,0
466,898,78.25,az,y,e,c,d,d,j,j,...,0,0,0,0,0,0,1,0,0,0
3092,6214,92.18,y,w,ae,c,d,q,i,c,...,1,0,0,0,0,0,0,0,0,0
3772,7558,91.92,y,r,ak,f,d,v,i,b,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.metrics import r2_score

r2 = Scorer(name="r2",
            scorer=r2_score,
            greater_is_better=True,
            extra_params = {})

preprocessor = PreprocessingTool(val_folds = True,
                                 n_folds = 5,
                                 seed = 42,
                                 prob_type="regression")

train_dataset = TabularDataset(train_df,
                               label = "y",
                               preprocessor = preprocessor,
                               type = "train")
eval_dataset = TabularDataset(eval_df,
                              label = "y",
                              preprocessor = preprocessor,
                              type = "eval")
test_dataset = TabularDataset(test_df,
                              preprocessor = preprocessor,
                              type = "infer")

trainer = Trainer(train_dataset,
                  eval_dataset = eval_dataset,
                  eval_metric = r2,
                  models = [
                    #   "LGB", "XGB", "CAT", "RF",
                             "SGD_LINEAR"],
                  early_stopping_rounds = 100,
                  of_mitigation_level = 0.2,
                  use_gpu = True,
                  use_cuda = True,
                  n_trials = 1000,
                  timeout = 100,
                  meta_timeout = 120,
                  seed = 42,
                  select_top = 3,
                  train_meta = True)

trainer.train()

LABEL: y
Problem type: regression. 
-----------------------------------------------------------------------------------------------------------------------------------------
IMPUTER:

Done. 
-----------------------------------------------------------------------------------------------------------------------------------------
DTYPES DETECTOR:


Found 2 unique raw np.dtype(s): [dtype('int64') dtype('O')].
Dropped 1 column(s) with index ID / non-categorical text features.
Converted 0 column(s) to numeric types.
Remaining features: 368 int feat(s), 0 float feat(s), 8 categorical feat(s).
-----------------------------------------------------------------------------------------------------------------------------------------
5-FOLD STRATIFIEDKFOLDREG TRAIN-VAL SPLITTER:

Done.
-----------------------------------------------------------------------------------------------------------------------------------------
SCALER:

Done.
---------------------------------------------------------------

  0%|          | 0/1000 [00:00<?, ?it/s]

Trial 0 finished with values: {'mean_train_r2': 0.5490930384595295, 'mean_val_r2': 0.506927606152111, 'optimized_metric': 0.49849451969062725},
and parameters: {'alpha': 0.02131306913881721, 'penalty': 'elasticnet', 'l1_ratio': 0.39252970387691866, 'learning_rate': 'invscaling', 'eta0': 0.0516316248469308, 'power_t': 0.24839752351839792, 'max_iter': 2673, 'tol': 5.2007140054830965e-05, 'shuffle': True, 'early_stopping': True, 'n_iter_no_change': 10, 'n_estimators_': None}.

Trial 1 finished with values: {'mean_train_r2': 0.6076520237393023, 'mean_val_r2': 0.5198794126985333, 'optimized_metric': 0.5023248904903795},
and parameters: {'alpha': 1.8892050017882176e-06, 'penalty': 'l2', 'l1_ratio': 0.4577861486664674, 'learning_rate': 'adaptive', 'eta0': 0.003915193249839491, 'power_t': 0.23429052149171828, 'max_iter': 1732, 'tol': 2.5550370565637377e-05, 'shuffle': False, 'early_stopping': True, 'n_iter_no_change': 4, 'n_estimators_': None}.

Trial 2 finished with values: {'mean_train_r2': 

,id,mean_train_r2,mean_val_r2,optimized_metric,params_alpha,params_early_stopping,params_eta0,params_l1_ratio,params_learning_rate,params_max_iter,...,params_penalty,params_power_t,params_shuffle,params_tol,params_n_estimators_,datetime_start,datetime_complete,duration,state,model
0,SGD_LINEAR_17,0.548574,0.538024,0.535914,0.075630,True,0.000365,0.544165,invscaling,1821,...,l1,0.123631,False,0.000013,None,2024-11-09 23:40:42.199798,2024-11-09 23:41:05.752819,0 days 00:00:23.553021,COMPLETE,"SGDRegressor(alpha=0.07562997659987251, early_..."
1,SGD_LINEAR_18,0.546207,0.537333,0.535559,0.080832,True,0.000114,0.514476,constant,1117,...,l1,0.126628,False,0.000013,None,2024-11-09 23:41:05.840389,2024-11-09 23:41:24.100276,0 days 00:00:18.259887,COMPLETE,"SGDRegressor(alpha=0.08083234398028248, early_..."
2,SGD_LINEAR_16,0.563832,0.533909,0.527925,0.019594,True,0.000172,0.496584,invscaling,1012,...,l1,0.154095,False,0.000043,None,2024-11-09 23:40:20.397184,2024-11-09 23:40:42.114881,0 days 00:00:21.717697,COMPLETE,"SGDRegressor(alpha=0.019594009373257167, early..."


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


TRAINING ELASTICNET META-LEARNER ...

Retraining best found models on each fold and generating oof preds ...
Training meta-learner using Optuna...
Done:
Number of models tried by Optuna: 2334.
Best ElasticNet parameters: {'alpha': 1.8734553555824869, 'l1_ratio': 0.00027992802761392637, 'max_iter': 9006}
Best r2 score: 0.538446219129101
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

...

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


Generated ensemble model. To use it, call .predict() / .predict_proba() method on trainer object.


VotingRegressor(estimators=[('SGD_LINEAR_17',
                             SGDRegressor(alpha=0.07562997659987251,
                                          early_stopping=True,
                                          eta0=0.0003649761970297234,
                                          l1_ratio=0.5441650521239655,
                                          max_iter=1821, n_iter_no_change=9,
                                          penalty='l1',
                                          power_t=0.12363123460604121,
                                          shuffle=False,
                                          tol=1.3497244558710712e-05)),
                            ('SGD_LINEAR_18',
                             SGDRegressor(alpha=0.08083234398028248,
                                          early_stopping=Tru...
                                          tol=1.2676137465545532e-05)),
                            ('SGD_LINEAR_16',
                             SGDRegressor(alpha=0.019594009373257167,
                                          early_stopping=True,
                                          eta0=0.00017220271750865017,
                                          l1_ratio=0.49658447738848466,
                                          max_iter=1012, n_iter_no_change=10,
                                          penalty='l1',
                                          power_t=0.15409543853623678,
                                          shuffle=False,
                                          tol=4.3419465505085236e-05))],
                weights=0    0.335072
1    0.334850
2    0.330077
Name: optimized_metric, dtype: float64)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


External Evaluation LEADERBOARD:


,id,model,custom_r2,mae,mse,rmse,r2
0,VotingEnsemble,"VotingRegressor(estimators=[('SGD_LINEAR_17',\...",0.577858,5.404676,65.706320,8.105943,0.577858
1,SGD_LINEAR_16,"SGDRegressor(alpha=0.019594009373257167, early...",0.576708,5.432372,65.885315,8.116977,0.576708
2,MetaLearner,"ElasticNet(alpha=1.8734553555824869, l1_ratio=...",0.575928,5.347383,66.006828,8.124459,0.575928
3,SGD_LINEAR_17,"SGDRegressor(alpha=0.07562997659987251, early_...",0.573107,5.421377,66.445905,8.151436,0.573107
4,SGD_LINEAR_18,"SGDRegressor(alpha=0.08083234398028248, early_...",0.572246,5.459214,66.579913,8.159652,0.572246


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


Retraining best found models on whole available data...
Done. Trainer is ready for inference and saved at path: saved_trainer.pkl
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 




In [ ]:
import dill
with open("tr.pkl", "wb") as f:
    dill.dump(trainer, f)

In [19]:
with open("tr.pkl", "rb") as f:
    new_tr = dill.load(f)

In [21]:
processed_test_data = test_dataset.process()
processed_test_data

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4206,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4207,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
preds = trainer.predict(processed_test_data, mode="ensemble")
preds

array([ 78.72376718,  94.46495028,  78.8818638 , ...,  92.88526377,
       111.10183589,  92.96680061])

In [10]:
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "y": preds
})
submission

,ID,y
0,1,78.723767
1,2,94.464950
2,3,78.881864
3,4,77.956778
4,5,112.631084
...,...,...
4204,8410,103.391248
4205,8411,93.705470
4206,8413,92.885264
4207,8414,111.101836


In [11]:
submission.to_csv("submission_meta.csv", index=False)

a
b
d


In [43]:
lista

['a', 'b', 'c']